# FINAL ASSIGMENT

### Instructions

Follow the example from [**Bootcamp 4**](https://www.youtube.com/watch?v=kHJLwfDUSWI&list=PL_wGNAk5B0pXq98BJBKsbNH2Qjp1lk8dG&index=32) for creating the quantum walk operator for the case of a circle with 4 nodes, and design the quantum walk operator for the case of a line with 16 nodes 

<img title="a title" alt="Alt text" src="Classiq_FinalAssignment_Fig1.png">

A. Create a well-detailed Python Jupyter notebook that explains your algorithm, including the code parts covered in class, and pictures/figures where reevant. 

1. Utilize the Python code from class: [*quantum walk circle example.py*](https://github.com/Classiq/classiq-library/blob/main/community/womanium/assignments/quantum_walk_circle_example.py). It can be found directly also in the Classiq Git Library in the [*community/womanium/assignments*](https://github.com/Classiq/classiq-library/tree/main/community/womanium/assignments) folder.
2. Fell free to extend the example beyond the requirements here and what was covered in class.

B. Contribute your notebook to the [Classiq Git Library](https://github.com/Classiq/classiq-library) to the folder [**community/womanium/assigments**](https://github.com/Classiq/classiq-library/tree/main/community/womanium/assignments)

1. Follow the [contribution guidelines](https://github.com/Classiq/classiq-library/blob/main/CONTRIBUTING.md)in order to contribute - NO need to opne an issue for this, you can directly open a PR
2. The PR title should be: Womanium Final Assignment - $<$First Name$>$ $<$Last Name$>$.
3. The file name should be in the following format: $<$**first_name$>$_$<$last_name$>$_hw4.ipynb**

### Implementation

As in the [**Bootcamp 4**](https://www.youtube.com/watch?v=kHJLwfDUSWI&list=PL_wGNAk5B0pXq98BJBKsbNH2Qjp1lk8dG&index=32) we have two important aspects to cover: the encoding, and the evolution. Two of the most important differences with the bootcamp is the number of vertices and the arrangement. So the $V$ set now will be

$$ V = \{0, 1, 2,..., 14, 15\}$$

On the other hand the encoding of the `adjacent_vertices` will be different, because the `adjacent_vertices` of the $0$ and $15$ is only one because is not closed. 

As covered in the bootcamp we need to implement the two operators $C$ and $S$

\begin{equation}
C= \sum_{j\in V} \vert j\rangle\langle j \vert \otimes (2\vert \partial_{j}\rangle\langle \partial_{j}\vert-\mathbb{I})
\end{equation}

\begin{equation}
S= \sum_{(j,k)\in E} \vert j, k\rangle\langle k,j \vert
\end{equation}

In [1]:
!pip install -U classiq

In [5]:
from classiq import *
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences

size = 4 

##############################################
############# Phase Kickback #################
###########3##################################
@qfunc
def prepare_minus(x:QBit):
#Here we have a qfunc that prepares/creates the minus state. 
#It receives a and qubit x and returns the minus state.
    X(x)
    H(x)
    
@qfunc
def diffuzer_oracle(aux: Output[QNum], x:QNum):
    #Oracle for the phase kickback
    aux^+(x!=0)
    #auxiliary qubit to the x state
    
@qfunc 
def zero_diffuzer(x: QNum):
#We have a qfunc that we call zero diffuzer and it applies only on a QNum (quantum variable) , so it receives x    
    aux = QNum('aux')
    #Then it initializes another auxiliary qubit for the phase kickback 
    allocate(1, aux)
    within_apply(compute=lambda: prepare_minus(aux), #Here we prepare the minus state on the aux qubit
                 action=lambda: diffuzer_oracle) #Here we apply the oracle that we want to have
    #and we use thewithin aplpy operation for actually applying the PK  
    
##############################################
############### C Operator ###################
##############################################

def W_iteration(i:int, vertices: QNum, adjacent_vertices:QNum):
    #Because we have a linear arrange we need to set like boundary conditions
    prob = [0]*(2**size)  # Here we initialize probability vector for 16 nodes
    if i == 0:
        prob[i + 1] = 1.0  # Here we set the first boundary condition if we are ath the first node, then 
        #the probability to move is one to the right node (node 1), e.i. we can only move to the right.
    elif i == (2**size) - 1:
        prob[i - 1] = 1.0  # As in the last condition, if we are at the last node, the only choice is
        #to move to the left (node 14)
    else:
        #In any other case we have 50/50 chance to move to the left or to the right.
        prob[i - 1] = 0.5  # Probability of moving to the left
        prob[i + 1] = 0.5  # Probability of moving to the right 
    #The rest of this function is the same as the example
    print(f'State={i}, prob vec ={prob}')
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
        #In the example given in class this is the repeat fuction in qmode language
        #This iterates over all vertices
        W_iteration(i, vertices, adjacent_vertices)

@qfunc
def edge_oracle(res: Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    #The oracle is the function that helps us to see if the vertex is an adjacent vertex.
    res |= (((vertices - adjacent_vertices) == 1) | ((vertices - adjacent_vertices) == -1))

        
@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
    #As in the example this swaps the state to the adjacent
    repeat(count= x.len,
        iteration= lambda i: SWAP(x[i],y[i]))
    
@qfunc
def S_operator(vertices: QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res, vertices, adjacent_vertices)
    control(ctrl= res==1,
            operand= lambda: bitwise_swap(vertices, adjacent_vertices))


In [6]:
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    # ENCODING
    allocate(size, vertices)
    hadamard_transform(vertices)
    allocate(size, adjacent_vertices)
    # EVOLUTION
    W_operator(vertices, adjacent_vertices)
    S_operator(vertices, adjacent_vertices) 

In [7]:
# Create and Synthesize
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0

The current version of 'classiq' has been deprecated, and will not be supported as of 2024-07-27. Please run "pip install -U classiq" to upgrade the classiq SDK to the latest version.


Opening: https://platform.classiq.io/circuit/00a6c9d2-f3da-40dd-a93d-9496a972e948?version=0.42.2


Here we can see the circuit 

<img title="a title" alt="Alt text" src="final6_12.jpg">

In [ ]:
write_qmod(create_model(main), "Quantum_walk_straightline")